## Performing openai native function calling with the langchain API
Langchain provides a cleaner interface for function calling in langchain, this allows us to more easily compose functions together for our applications

Pydantic data classes are like regular python classes but with the validation power of pydantic. This allows us to have more type safe and usage safe python applications

In [1]:
from pydantic import BaseModel, Field
from typing import List


class User(BaseModel):
    name: str
    age: int
    email: str

In [2]:
foo_p = User(name="Jane", age=32, email="jane@gmail.com")

In [4]:
foo_p.name, foo_p.age, foo_p.email

('Jane', 32, 'jane@gmail.com')

In [5]:
foo_p = User(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing

In [6]:
class Class(BaseModel):
    students: List[User]

In [7]:
obj = Class(
    students=[User(name="Jane", age=32, email="jane@gmail.com")]
)

In [8]:
obj.students

[User(name='Jane', age=32, email='jane@gmail.com')]

### Defining OpenAI functions using the pydantic API library
We can use the type information in pydantic to generate the list of functions to be sent to OpenAI 

In [9]:
class WeatherSearch(BaseModel):
    """
    Call this with an airport code to get the weather at that airport
    """
    airport_code:str = Field(description="airport code to get weather for")

In [10]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [11]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [13]:
from pprint import pprint

In [14]:
pprint(weather_function)

{'description': 'Call this with an airport code to get the weather at that '
                'airport',
 'name': 'WeatherSearch',
 'parameters': {'description': 'Call this with an airport code to get the '
                               'weather at that airport',
                'properties': {'airport_code': {'description': 'airport code '
                                                               'to get weather '
                                                               'for',
                                                'title': 'Airport Code',
                                                'type': 'string'}},
                'required': ['airport_code'],
                'title': 'WeatherSearch',
                'type': 'object'}}


In [17]:
class WeatherSearch1(BaseModel): # would fail because no description added to the function name 
    airport_code: str = Field(description="airport code to get weather for")

In [18]:
convert_pydantic_to_openai_function(WeatherSearch1)

KeyError: 'description'

In [19]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str # would generate the function without proper description for the airport code field

In [20]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'description': 'Call this with an airport code to get the weather at that airport',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code'],
  'title': 'WeatherSearch2',
  'type': 'object'}}